## tensorflow 2.0 에서 바뀐 것들
- API 정리
    - 흩어져있던 다양한 api들을 하나로 통합, 잘 사용하지 않는 것들 제거
- 이거 모드 (eager mode)
    - 기존에는 텐서플로우 api를 이용해 그래프를 만든 후 별도로 세션을 통해 해당 그래프를 실행(session.run())
    - 2.0 부터는 파이썬과 동일한 이거모드로 실행 => 연산을 구성하면서 바로바로 값을 확인 가능
- 전역 메커니즘 제거
- 세션을 대신하는 함수
- 케라스를 활용한 모델 구축을 권장
    - Sequential API
    - Functional API
    - Functional/Sequential API
        - (+) Custom Layers
    - Subclassing (Custom Model) - 이 책에서는 주로 이것을 이용

## Keras Sequential

레이어들을 순차적으로 연결하는 스택 구조. 구현 간단하지만 다음 경우 사용 불가
- 다중 입력값 모델
- 다중 출력값 모델
- 고유 층을 활용하는 모델
- 데이터 흐름이 순차적이지 않은 모델

위 경우는 Functional API를 사용하자

In [3]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

## Keras Functional API

입력값을 받는 Input 모듈을 선언해야함. (입력 받는 형태의 shape를 인자로 받음)

In [4]:
inputs = tf.keras.Input(shape=(32,))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
prediction = layers.Dense(10, activation='softmax')(x)

## Keras Custom Model

- 사용자 정의 layer.

- 새로운 연산을 하거나 편의를 위해 여러 레이어를 하나로 묶은 레이어를 만들고 싶을 때

- tf.kears.layers.Layer 를 상속받음
    - 3개의 메소드 정의! (init, build, call)


In [9]:
# __init__, build, call 정의해야함

class CustomLayer(layers.Layer):
    def __init__(self, hidden_dimension, hidden_dimension2, output_dimension):
        # 하이퍼 파라미터 설정
        self.hidden_dimension = hidden_dimension
        self.hidden_dimension2 = hidden_dimension
        self.output_dimension = output_dimension
        super(CustomLayer, self).__init__()
    
    def build(self, input_shape):
        # 가중치와 관련된 값들
        self.dense_layer1 = layers.Dense(self.hidden_dimension, activation='relu')
        self.dense_layer2 = layers.Dense(self.hidden_dimension2, activation='relu')
        self.dense_layer3 = layers.Dense(self.output_dimension, activation='softmax')
        
    def call(self, inputs):
        # 해당 층의 로직 정의
        x = self.dense_layer1(inputs)
        x = self.dense_layer2(x)
        
        return self.dense_layer3(x)


In [11]:
# 커스텀 레이어 사용
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(CustomLayer(64, 64, 10))

## Subclassing(Custom Model)
- 가장 자유도가 높음. 
- 이 책에서 가장 많이 사용
- tf.keras.Model 상속
    - 2개 메서드 구현 (init, call)

In [12]:
class MyModel(tf.keras.Model):
    def __init__(self, hidden_dimension, hidden_dimension2, output_dimension):
        super(MyModel, self).__init__(name='my model')
        self.dense_layer1 = layers.Dense(hidden_dimension, activation='relu')
        self.dense_layer2 = layers.Dense(hidden_dimension2, activation='relu')
        self.dense_layer3 = layers.Dense(output_dimension, activation='softmax')
    
    def call(self, inputs):
        x = self.dense_layer(inputs)
        x = self.dense_layer2(x)
        return self.dense_layer3(x)
        

## 모델 학습 (검증, 예측 포함)

모델 학습 방법
1. 케라스 모델의 내장 API 사용 (model.fit(), model.evaluate(), model.predict())
2. 학습, 검증, 예측 등 모든 과정을 GradientTape 객체를 활용해 직접 구현하는 방법

앞의 모델 구축에서 만든 모델은 케라스의 모델 객체이기 때문에 여러 메서드가 이미 내장되어있음 

__먼저 해야할 일은 손실함수, 옵티마이저, metric 등을 지정해줘야함__

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.CategoricalCrossentropy(),
             metrics=[tf.keras.metrics.Accuracy()])

## 위와 동일 (문자열 형태로 옵티마이저, 손실함수, 메트릭 지정)

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
# 정의된 모델 객체를 대상으로 학습, 평가, 예측 메서드를 호출하면 compile로 정의된 값들을 활용해 학습 진행
model.fit(x_train,
         y_train,
         batch_size=64,
         epochs=3)


In [ ]:
# 학습 과정에서 각 에폭마다 검증을 진행하기
# evaluate 메서드를 사용해 검증도 가능하지만, 매번 에폭을 호출해야하므로 fit 함수에 검증 데이터를 추가로 넣자
model.fit(x_train,
         y_train,
         batch_size=64,
         epochs=3,
         validation_data(x_val, y_val))